In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='../custom.css'></head>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

import matplotlib as mpl
import matplotlib.colors as mcolors  
import matplotlib.pyplot as plt      

from matplotlib import rc 

In [3]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b k_B}`
        
    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        
    Returns:
        p_c: Critical pressure in Pa.\n
        v_c: Critical volume in m^3.\n
        T_c: Critical tenperature in K.\n
        
    """
    
    k_B = 1.3806488e-23
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/k_B
    
    return p_c, v_c, T_c

In [4]:
def isotherms(a, b, T_range, v_range):
    
    """
    
        This function calculates the p(v, T) plane
        from given a and b parameters of 
        the Van der Waals equation of state for real gases.

    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        T_range: Tuple containing maximum and minimum values of
        temperature to be plotted. Temperatures must be expressed
        in terms of the critical temperature.\n
        v_range: Tuple containing maximum and minimum values of
        volumen to be plotted. Volumens must be expressed
        in exponents of the critical volumen (v in 10^v_range).\n
        
    Returns:
        isotherms: a dictionary containing the values of v and the
        pressures of the isotherms in the following form:\n
            isotherms['v'] = np.array containing the values of v
            in a logarithmic scale.\n
            isotherms['value of the isotherm'] = np.array containing
            the values of p.\
    """
    
    isotherms = {}
    
    p_c, v_c, T_c = calculate_critic(a, b)
    v_R = np.logspace(min(v_range), max(v_range), num = 1000, base = 10.0)

    isotherms['v'] = v_R

    for T in T_range:
        p_R = []
        for v in v_R:
            val = 8.0/3.0*T/(v - 1.0/3.0) - 3.0/v**2
            p_R = np.append(p_R, val)
        
        isotherms['%.2f' % T] = p_R

    return isotherms

In [5]:
def update_isotherm(change):
    
    """    
        This function makes the isotherm expressed by a
        float ipython widget visible in the 'fig' plot and
        hides the rest of the lines (the ones in the list 
        'saved' are always displayed).\n

    Args:
        change: variable returned by my_float_widget.observe(update
        _isotherm, 'value').\n

    """
    
    old, = np.where(t_range == float(change.old))[0]
    i, = np.where(t_range == float(change.new))[0]
    
    if old not in saved:
        fig.marks[old].visible = False
        fig.marks[old].display_legend = False
        
    fig.marks[i].visible = True
    fig.marks[i].display_legend = True

In [6]:
def save_isotherm(a):
    
    """    
        This function adds the isotherm expressed by a
        float ipython widget ('T_r') visible to the 'saved' list
        (the ones in the list 'saved' are always displayed
        in the 'fig' plot).\n

    Args:
        a: variable returned by save_button.on_click(save_isotherma)
        .\n

    """
    
    i, = np.where(t_range == float(T_r.value))[0]
    saved.append(i)

In [7]:
def undo(change):
    
    """    
        This function pops the isotherm expressed by a
        float ipython widget ('T_r') visible from the 'saved' list
        (the ones in the list 'saved' are always displayed
        in the 'fig' plot).\n

    Args:
        change: variable returned by undo_button.on_click(undo)
        .\n

    """
        
    if len(saved) > 0:
        popped = saved.pop(-1)
        i, = np.where(t_range == float(T_r.value))[0]
        if i != popped:
            fig.marks[popped].visible = False

In [8]:
def matplotlib_figure(pdf, png, fig):

    """
    
        This function calculates makes a copy of the bqplot Figure
        'fig' and creates a png/pdf of it using matplotlib. The name
        of the file is taken from 'png_name' of 'pdf_name' ipython
        text widgets (depending on the export format).\n

    Args:
        pdf: Boolean. If True the exported file will be a .pdf file.\n
        png: Boolean. If True the exported file will be a .png file.\nb: Term related with the volume that is occupied by one 
        fig: bqplot Figure to be exported.\n
        
    Returns:
        save_filename: string with the name of the file created.\n
        
    """
    
    
    plt.ioff()

    lines = []
    for line in fig.marks:
        
        if line.visible == True:
            lines.append(line)

    ax = fig.axes[0]
    ay = fig.axes[1]

    # irudiaren diseinuerako
    plt.rc('text', usetex=True)
    mpl.rcParams['errorbar.capsize'] = 3
    colors_ = list(mcolors.TABLEAU_COLORS.values())

    # irudiko eskalak definitzeko: hobetu egin behar da, balioen eta tarteen arabera egokiak diren tick-en kopurua finkatzeko, beti kopuru txikia izanik  

    # irudia
    fig_pdf, ax0 = plt.subplots(nrows=1, ncols=1, figsize=(16, 12), );
    plt.subplots_adjust(left=0.25, bottom=0.25, right=0.9, top=None, wspace=0.0, hspace=0.0)
    ax0.tick_params(axis='both', labelsize=30, pad=10, length=12)

    ax0.set_xlim(ax.scale.min, ax.scale.max)
    ax0.set_ylim(ay.scale.min, ay.scale.max)
    
    ax0.xscale = 'log'
    
    # ticks
    ax0.yaxis.set_major_locator(mpl.ticker.NullLocator());
    ax0.xaxis.set_major_locator(mpl.ticker.NullLocator());

    #plt.xticks()

    #ax0.yaxis.set_major_locator(plt.MaxNLocator(nbins=ay.num_ticks));
    #ax0.xaxis.set_major_locator(plt.MaxNLocator(nbins=ax.num_ticks));

    # etiketak
    if len(fig.title.split()) > 0: 
        ax0.set_title(r'\textrm{%s}' % fig.title, size=35, pad=20)
    ax0.set_xlabel(r'\textrm{%s}' % ax.label, size=30, labelpad=15)
    ax0.set_ylabel(r'\textrm{%s}' % ay.label, size=30, labelpad=15)

    # datuak irudikatu
    for line in lines:
        str_to_rgb = [float(c)/255.0 for c in line.colors[0][4:-1].split(',')]
        color = tuple(str_to_rgb)
        ax0.semilogx(
            line.x,
            line.y,
            label=r'\textrm{%s}' % line.labels[0],
            color=color,
            );

    # egokia den ordenean legendakoak adierazteko
    handles, labels = plt.gca().get_legend_handles_labels()

    order = [i for i in range(len(lines))]

    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])
    ax0.legend(bbox_transform=ax0.transAxes,
               #bbox_to_anchor=(0.95, 0.30),
               loc='best',
               ncol=1,
               borderaxespad=1.5,
               frameon=False,
               fontsize=25);

    # irudia gorde
    if pdf:
        save_filename = pdfName.value

        if len(save_filename.split()) == 0:
            save_filename = 'pdf_file'

        if '.pdf' not in save_filename:
            save_filename = save_filename + '.pdf'

        plt.savefig(save_filename, format='pdf', dpi=150, bbox_inches="tight");

    if png:
        save_filename =  png_name.value#png_name.value
        if len(save_filename.split()) == 0:
            save_filename = 'png_file'

        if '.png' not in save_filename:
            save_filename = save_filename + '.png'
        plt.savefig(save_filename, format='png', dpi=150, bbox_inches="tight");
        png_name.value = ''
    return (save_filename)

In [9]:
def save_png(change):
    
    """
    
        This function calls the matplotlib_figure() function
        and if the is no errors creates an HTML button to download
        the file.\n

    Args:
        change: variable returned by export_png_button.on_click(save_png).\n
        
    """
    
    try:
        png_created_text.value ="<b>Creating PNG file...</b>"
        filename = matplotlib_figure(pdf = False, png = True, fig = fig)
        png_created_text.value = "<span style='color:green'><b>PNG file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download PNG</button></form>"

    except:
        png_created_text.value = "<span style='color:red'><b>An error has occurred</b></span>"

In [12]:
"""

.. module:: p_v_2D.ipynb
    :sypnopsis: This module creates an interface to interact with the
    Van der Waals isotherms.\n

.. moduleauthor:: Jon Gabirondo López (jgabirondo001@ikasle.ehu.eus)

"""

t_limits = (0.2, 3.0)
v_limits = (-0.4, 2.0) #v in 10^v_limits range

temp = np.arange(t_limits[0], t_limits[1]+0.05, 0.05)
t_range = np.around(temp, 2)
T_c_index, = np.where(t_range == 1.0)[0]
data = isotherms(5.537,0.03049, t_range, v_limits)
saved = []


scale_x = bqs.LogScale(min = min(data.get('v')), max = max(data.get('v')))
scale_y = bqs.LinearScale(min = min(data.get('1.00'))-0.3, max = 3.0)

fig = bq.Figure(title='Van der Waals isotherms',
                marks=[],
                axes=[],
                animation_duration=500,
                layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=75, right=30)
    )

axis_x = bqa.Axis(scale=scale_x,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                ticks=[0,10,100],
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v [log]',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(scale=scale_y,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig.axes = [axis_x, axis_y]
isotherm_lines = []
size = len(data.keys()) - 1
color = 250
h = 0
step = int(np.around([color/size], 0)[0])

for t in data.keys():
    if t != 'v':
        isotherm_lines.append(bqm.Lines(
            x = data.get('v'), 
            y = data.get(t), 
            scales = {'x': scale_x, 'y': scale_y}, 
            visible = t == '1.00',
            colors = ['rgb(255,'+str(h)+',0)'],
            labels = ['T = ' + t],
            ))
        h = h + step

fig.marks = isotherm_lines
fig.marks[T_c_index].visible = True
fig.marks[T_c_index].display_legend = True

T_r = widgets.FloatSlider(
        value=1.0,
        min=0.4,
        max=3.0,
        step=0.05,
        description=r'\( \frac{T}{T_c} \)',
        disabled=False,
        continuous_update=True,
        orientation='vertical',
        readout=True,
        readout_format='.2f',
        layout = widgets.Layout(height = '80%', margin = '45px 0 0 0')
    )

T_r.observe(update_isotherm, 'value')

save_button = widgets.Button(
        description='Save isotherm',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
    )

save_button.on_click(save_isotherm)

undo_button = widgets.Button(
        description='Undo',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
    )

undo_button.on_click(undo)

png_name = widgets.Text(
    value='',
    placeholder="Save as (optional):",
    disabled = False,
)

export_png_button = widgets.Button(
        description='Export as PNG',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
    )

png_created_text = widgets.HTML(
    value = "",
    )

export_png_button.on_click(save_png)

widgets.VBox([
    widgets.HBox([fig,
                  T_r,
                  widgets.VBox(
                      [save_button, undo_button,],
                      layout = widgets.Layout(margin = '20% 0 0 0') )

                 ], layout = widgets.Layout(width = '100%')),
    widgets.HBox([png_name, export_png_button,], layout = widgets.Layout(align_self = 'center',
                                                                        margin_bottom = '15px')),
    widgets.HBox([png_created_text], )])